# Chapter 8: Data Wrangling: Join, Combine and Reshape

In [1]:
import numpy as np
import pandas as pd #Most common alias

# 8.1 Hierarchical Indexing
## 8.1.1 Introduction
Hierarchical indexing is an important feature of pandas that enables you to have multiple
(two or more) index levels on an axis. Indices are passed in the form of nested lists from higher to level lower level. There should be same number of indices as the number of elements.

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -1.822038
   2    1.457854
   3    1.031367
b  1    0.925565
   3   -0.882284
c  1   -1.763864
   2    0.073043
d  2    0.380293
   3    1.062616
dtype: float64

index attribute of dataaframe shows all levels of indices with their respective codes. For example there are two levels here: 
- ['a', 'b', 'c', 'd']
- [1, 2, 3]

First element has [a][1] index. Both of them are 0th element of their respecctive lists so the index-wise code for this element is 0,0 as shown in list of codes. Where we have 0 in the staarting of both nested lists
 

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

With a hierarchically indexed object, partial indexing is possible, enabling
you to concisely select subsets of the data:

In [4]:
data['b']

1    0.925565
3   -0.882284
dtype: float64

In [5]:
data['b':'c']

b  1    0.925565
   3   -0.882284
c  1   -1.763864
   2    0.073043
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1    0.925565
   3   -0.882284
d  2    0.380293
   3    1.062616
dtype: float64

Selection is even possible from an “inner” level:

In [7]:
data.loc[:, 2]

a    1.457854
c    0.073043
d    0.380293
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based
operations like forming a pivot table. For example, you could rearrange the data into
a DataFrame using its unstack method.

For those unaware of unstack(), it unstacks lowest index in the form of other axis from a dataframe. As in our case, it becomes column of the dataframe. Non-correesponding entities are filled with NaN vaalue.

In [8]:
data.unstack()

,1,2,3
a,-1.822038,1.457854,1.031367
b,0.925565,NaN,-0.882284
c,-1.763864,0.073043,NaN
d,NaN,0.380293,1.062616


Further unstacking will make the column axis the highest row axis index. In this way different levels of indices are interchanged.

In [9]:
data.unstack().unstack()

1  a   -1.822038
   b    0.925565
   c   -1.763864
   d         NaN
2  a    1.457854
   b         NaN
   c    0.073043
   d    0.380293
3  a    1.031367
   b   -0.882284
   c         NaN
   d    1.062616
dtype: float64

The inverse operation of unstack is stack effectively cancelling its changes and restacking datafraame again

In [10]:
data.unstack().stack()

a  1   -1.822038
   2    1.457854
   3    1.031367
b  1    0.925565
   3   -0.882284
c  1   -1.763864
   2    0.073043
d  2    0.380293
   3    1.062616
dtype: float64

Ignore if there is a confusion in them :) stack and unstack will be explored in more detail later in this chapter.

### Back toHierarchical Indexing
With a DataFrame, either axis can have a hierarchical index. It is to be noted that index argument accepts value for rows whereas columns aaccepts for columns

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
columns=[['Ohio', 'Ohio', 'Colorado'],
['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects). If so, these
will show up in the console output:

In [12]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [13]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

A MultiIndex can be created by itself and then reused; the columns in the preceding
DataFrame with level names could be created like this:

In [14]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### 8.1.2 Reordering and Sorting Levels
At times you will need to rearrange the order of the levels on an axis or sort the data
by the values in one specific level. The swaplevel takes two level numbers or names
and returns a new object with the levels interchanged (but the data is otherwise
unaltered):

In [15]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

Similaarly, sort_index sorts the data using only the values in a single level.
When swapping levels, it’s not uncommon to also use sort_index so that the result is
lexicographically sorted by the indicated level:

In [16]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

Putting them together

In [17]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 8.1.3 Summary Statistics by Level
DataFrame and Series have a level
option in which you can specify the level you want to aggregate by on a particular
axis. Consider the above DataFrame; we can aggregate by level on either the rows or
columns like so:

In [18]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [19]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 8.1.4 Indexing with a DataFrame’s columns
You can use one or more columns from a DataFrame as the row
index; alternatively, you may wish to move the row index into the DataFrame’s columns.
Here’s an example DataFrame:

In [20]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
'c': ['one', 'one', 'one', 'two', 'two','two', 'two'], 'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s set_index function will create a new DataFrame using one or more of
its columns as the index:

In [21]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default the columns are removed from the DataFrame, though you can leave them
in:

In [22]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index, on the other hand, does the opposite of set_index; the hierarchical
index levels are moved into the columns:

In [23]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining and Merging Datasets
Data contained in pandas objects can be combined together in a number of ways:
- pandas.merge connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations. Waarning: SQL revisit required for some topics 
- pandas.concat concatenates or “stacks” together objects along an axis.
- The combine_first instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

## 8.2.1 Database-Style DataFrame Joins
Merge or join operations combine datasets by linking rows using one or more keys.
These operations are central to relational databases (e.g., SQL-based). The merge
function in pandas is the main entry point for using these algorithms on your data.
Let’s start with a simple example:

In [24]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a many-to-one join; the data in df1 has multiple rows labeled a
and b, whereas df2 has only one row for each value in the key column. Calling merge
with these objects we obtain:

In [26]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Note that I didn’t specify which column to join on. If that information is not specified,
merge uses the overlapping column names as the keys. It’s a good practice to
specify explicitly, though:

In [27]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately:

In [28]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


By default merge does an 'inner' join which discards records with non-maatching keys and only keeping the keys in the result are the intersection,
or the common set found in both tables. Other possible options are 'left',
'right', and 'outer'. The outer join takes the union of the keys, combining the
effect of applying both left and right joins:

In [29]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


Different join types with how argument:
- 'inner' Use only the key combinations observed in both tables
- 'left' Use all key combinations found in the left table
- 'right' Use all key combinations found in the right table
- 'output' Use all key combinations observed in both tables together

In [30]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [31]:
pd.merge(df1, df2, how='inner') #default

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


To merge with multiple keys, pass a list of column names:

In [32]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})
 
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'], 'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


A last issue to consider in merge operations is the treatment of overlapping column
names. While you can address the overlap manually (see the earlier section on
renaming axis labels), merge has a suffixes option for specifying strings to append
to overlapping names in the left and right DataFrame objects:

In [33]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [34]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


Quick Summary:
- left: DataFrame to be merged on the left side.
- right: DataFrame to be merged on the right side.
- how: One of 'inner', 'outer', 'left', or 'right'; defaults to 'inner'.
- on: Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, will use the intersection of the column names in left and right as the join keys.
- left_on: Columns in left DataFrame to use as join keys.
- right_on: Analogous to left_on for left DataFrame.
- left_index: Use row index in left as its join key (or keys, if a MultiIndex).
- right_index: Analogous to left_index.
- sort: Sort merged data lexicographically by join keys; True by default (disable to get better performance in some cases on large datasets).
- suffixes: Tuple of string values to append to column names in case of overlap; defaults to ('_x', '_y') (e.g., if 'data' in both DataFrame objects, would appear as 'data_x' and 'data_y' in result).
- copy: If False, avoid copying data into resulting data structure in some exceptional cases; by default always copies.
- indicator: Adds a special column _merge that indicates the source of each row; values will be 'left_only', 'right_only', or 'both' based on the origin of the joined data in each row.

## 8.2.2 Merging on Index
In some cases, the merge key(s) in a DataFrame will be found in its index. In this
case, you can pass left_index=True or right_index=True (or both) to indicate that
the index should be used as the merge key:

In [35]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [36]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [37]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Since the default merge method is to intersect the join keys, you can instead form the
union of them with an outer join:

In [38]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Hierarchical Indexing + Multiple Key Merge

In [39]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio','Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)})

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [40]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],[2001, 2000, 2000, 2000, 2001, 2002]],
    columns=['event1', 'event2'])

righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In this case, you have to indicate multiple columns to merge on as a list (note the
handling of duplicate index values with how='outer'):

In [41]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [42]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


Using the indices of both sides of the merge is also possible:

In [43]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'], columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [44]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [45]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a convenient join instance for merging by index. It can also be used
to combine together many DataFrame objects having the same or similar indexes but
non-overlapping columns. In the prior example, we could have written:

In [46]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame’s join
method performs a left join on the join keys, exactly preserving the left frame’s row
index. It also supports joining the index of the passed DataFrame on one of the columns
of the calling DataFrame:

In [47]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as
an alternative to using the more general concat function described in the next
section:

In [48]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
            index=['a', 'c', 'e', 'f'],
            columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [49]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


## 8.2.3 Concatenating Along an Axis
Another kind of data combination operation is referred to interchangeably as concatenation,
binding, or stacking. NumPy’s concatenate function can do this with
NumPy arrays:

In [50]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [51]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

The concat function in pandas provides a consistent way for concatenation.

In [52]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [53]:
pd.concat([s1, s2, s3], axis=1)

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the sorted
union (the 'outer' join) of the indexes. You can instead intersect them by passing
join='inner':

In [54]:
s4 = pd.concat([s1, s3])

In [55]:
pd.concat([s1, s4], axis=1)

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [56]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


You can even specify the axes to be used on the other axes with join_axes:

In [57]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [58]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [59]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In the case of combining Series along axis=1, the keys become the DataFrame column
headers:

In [60]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [61]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
        columns=['one', 'two']) 
df1

,one,two
a,0,1
b,2,3
c,4,5


In [62]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
        columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [63]:
 pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

If you pass a dict of objects instead of a list, the dict’s keys will be used for the keys
option:

In [64]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

We can name the created axis levels with the names
argument:

In [65]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
names=['upper', 'lower'])

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [66]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
names=['upper', 'lower'])

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

A last consideration concerns DataFrames in which the row index does not contain
any relevant data:

In [67]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1


,a,b,c,d
0,0.526972,-0.166591,1.494389,-0.157992
1,-0.036899,0.959119,0.725565,-0.353652
2,-0.152968,-0.983751,-0.088603,2.321410


In [68]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,1.119379,0.454974,0.469310
1,-1.513295,-1.334750,0.971163


In [69]:
pd.concat([df1, df2], ignore_index=True)

c:\users\hamza\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,0.526972,-0.166591,1.494389,-0.157992
1,-0.036899,0.959119,0.725565,-0.353652
2,-0.152968,-0.983751,-0.088603,2.321410
3,0.469310,1.119379,NaN,0.454974
4,0.971163,-1.513295,NaN,-1.334750


## 8.2.4 Combining Data with Overlap
There is another data combination situation that can’t be expressed as either a merge
or concatenation operation. You may have two datasets whose indexes overlap in full
or part. As a motivating example, consider NumPy’s where function, which performs
the array-oriented equivalent of an if-else expression:

In [70]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
a,b

(f    NaN
 e    2.5
 d    NaN
 c    3.5
 b    4.5
 a    NaN
 dtype: float64, f    0.0
 e    1.0
 d    2.0
 c    3.0
 b    4.0
 a    5.0
 dtype: float64)

In [71]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

Series has a combine_first method, which performs the equivalent of this operation
along with pandas’s usual data alignment logic:

In [72]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

With DataFrames, combine_first does the same thing column by column, so you
can think of it as “patching” missing data in the calling object with data from the
object you pass:


In [73]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],'b': [np.nan, 2., np.nan, 6.], 'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.], 'b': [np.nan, 3., 4., 6., 8.]})

In [74]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [75]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [76]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 8.3 Reshaping and Pivoting
There are a number of basic operations for rearranging tabular data. These are alternatingly
referred to as reshape or pivot operations.
## 8.3.1 Reshaping with Hierarchical Indexing
Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
There are two primary actions:
stack
This “rotates” or pivots from the columns in the data to the rows
unstack
This pivots from the rows into the columns
I’ll illustrate these operations through a series of examples. Consider a small Data‐
Frame with string arrays as row and column indexes:

In [77]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
        index=pd.Index(['Ohio', 'Colorado'], name='state'),
        columns=pd.Index(['one', 'two', 'three'],name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


### Back to stack and unstack 
Using the stack method on this data pivots the columns into the rows, producing a
Series:

In [78]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

From a hierarchically indexed Series, you can rearrange the data back into a Data‐
Frame with unstack:

In [79]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default the innermost level is unstacked (same with stack). You can unstack a different
level by passing a level number or name:

In [80]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [81]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking maay introduce missing data if all of the values in the level aren’t found
in each of the subgroups:

In [82]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [83]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Stacking ignores missing data(however you can chhoose not to ignore it.... next example) so even in this case it reverts the effect of unstaacking

In [84]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [85]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

Remember: When you unstack in a DataFrame, the level unstacked becomes the lowest level in
the result:

In [86]:
df = pd.DataFrame({'left': result, 'right': result + 5},
        columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [87]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [88]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## 8.3.2 Pivoting: “Long” to “Wide” Format
Stacked format is a long format as stacking results in a lengthier dataframe with same data. This caan be done by stacking but most commonly is achieved by usinng pivot method.

In [89]:
data = pd.read_excel('pandas chapter 8 data.xlsx')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [90]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


This is the so-called long format for multiple time series, or other observational data
with two or more keys (here, our keys are date and item). Each row in the table represents
a single observation.
In some cases, the data may be more difficult
to work with in this format; you might prefer to have a DataFrame containing
one column per distinct item value indexed by timestamps in the date column. Data‐
Frame’s pivot method performs exactly this transformation:

In [91]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


How pivot method works?

The first two values passed are the columns to be used respectively as the row and
column index, then finally an optional value column to fill the DataFrame. Suppose
you had two value columns that you wanted to reshape simultaneously:

In [92]:
ldata['value2'] = np.random.randn(len(ldata))
ldata

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.813693
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.513183
2,1959-03-31 23:59:59.999999999,unemp,5.800,-1.801176
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.030342
4,1959-06-30 23:59:59.999999999,infl,2.340,-1.815042
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.663468
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.944044
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.477833
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.217916
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.616918


By omitting the last argument, you obtain a DataFrame with hierarchical columns (Revisit hierarchical indexing waay up in the notebook):

In [93]:
pivoted = ldata.pivot('date', 'item')
pivoted

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.513183 -0.813693   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.815042 -0.030342   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.477833 -0.944044   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  2.392660  0.616918   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.967992  0.213109   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.801176  
1959-06-30 23:59:59.999999999 -0.663468  
1959-09-30 23:59:59.999999999 -0.217916  
1959-12-31 23:59:59.999999999  1.054387  
1960-03-31 23:59:59.999999999 -0.597046

In [94]:
pivoted['value']

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


Note that pivot is equivalent to creating a hierarchical index using set_index followed
by a call to unstack:

In [95]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.513183 -0.813693   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.815042 -0.030342   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.477833 -0.944044   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  2.392660  0.616918   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.967992  0.213109   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.801176  
1959-06-30 23:59:59.999999999 -0.663468  
1959-09-30 23:59:59.999999999 -0.217916  
1959-12-31 23:59:59.999999999  1.054387  
1960-03-31 23:59:59.999999999 -0.597046

## 8.3.3 Melting: “Wide” to “Long” Format
An inverse operation to pivot for DataFrames is pandas.melt. Rather than transforming
one column into many in a new DataFrame, it merges multiple columns into
one, producing a DataFrame that is longer than the input. Let’s look at an example:

In [96]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
'A': [1, 2, 3],
'B': [4, 5, 6],
'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


The 'key' column may be a group indicator, and the other columns are data values.
When using pandas.melt, we must indicate which columns (if any) are group indicators.
Let’s use 'key' as the only group indicator here:

In [97]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Using pivot, we can reshape back to the original layout. 

Pivot and melt form same pair as stack and unstack but with more diversity and flexibility.

In [98]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


Since the result of pivot creates an index from the column used as the row labels, we
may want to use reset_index to move the data back into a column:

In [99]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


You can also specify a subset of columns to use as value columns:

In [100]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


pandas.melt can be used without any group identifiers, too. In this caase it results in series of variables and values

In [101]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [102]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


# End of the Chapter 8 :)
I tried to cover as much as I could and I have double checked that there is nothing left in the chapter. I think the most important topics are pivoting, melting and merge. Do them thoroughly. Plus every topic in this chapter is dependent on previous topics of the chapter so do not move on to next topic unless you are sure you have command over current topic.